In [1]:
# export CUDA_VISIBLE_DEVICES=2,5
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import pkg_resources
from datasets.downstream import pearl_kfold_dataset
from finetune_evaluator import Evaluator
from downstream import model_for_pearls
import torch
from torch import nn
from torchsummary import summary

/tmp/ipykernel_3746125/1336740768.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
def count_parameters(module: nn.Module) -> None:
    """
    Print the number of total and trainable parameters in a torch.nn.Module.
    """
    total_params = sum(p.numel() for p in module.parameters())
    trainable_params = sum(p.numel() for p in module.parameters() if p.requires_grad)

    print(f"Total parameters: {total_params}")
    print(f"Trainable parameters: {trainable_params}")

In [3]:
class Params:
    def __init__(self):
        self.datasets_dir = "/mnt/disk1/aiotlab/namth/EEGFoundationModel/datasets/pearl_30s_oldnumpy/108/fold_0"
        self.batch_size = 8
        self.use_pretrained_weights = True
        self.foundation_dir = "/mnt/disk1/aiotlab/namth/EEGFoundationModel/EEGPT/checkpoint/108.ckpt"
        self.dropout = 0.1
        self.cuda = 2
        self.modality_mode = 'multi'
        self.project_name = 'BigFaced'
        self.frozen = True
        self.num_of_classes = 2

params = Params()

loader = pearl_kfold_dataset.LoadDataset(params).get_data_loader()

train_eval = Evaluator(params, loader['train'])
val_eval = Evaluator(params, loader['val'])
test_eval = Evaluator(params, loader['test'])

502 119 131
752


In [4]:
model = model_for_pearls.Model(params).cuda()

In [5]:
x, y = next(iter(loader['train']))
x = (x[0].cuda(), x[1].cuda())
x[0].shape, x[1].shape, y.shape

(torch.Size([8, 19, 6000]), torch.Size([8, 28]), torch.Size([8]))

In [6]:
with torch.no_grad():
    out = model(x)
out.shape

torch.Size([8])

In [7]:
print('official model')
count_parameters(model)

official model
Total parameters: 2150781
Trainable parameters: 2150781
